<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Предсказание токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Подключим используемые библиотеки:

In [1]:
!pip install nltk -q

Импортируем необходимые пакеты:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd
import numpy as np
import re
import os
import pickle
import warnings

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings('ignore')
tqdm.pandas()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Загрузим исходный датасет:

In [3]:
def check_and_read(path, index_col=None, parse_dates=None):
    if os.path.exists(path):
        return pd.read_csv(path, index_col=index_col, parse_dates=parse_dates)
    else:
        print(f'Не найден файл по указанному пути: {path}')


df = check_and_read('datasets/toxic_comments.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Как видно, набор данных достаточно большой и содержит почти 160000 образцов текста.

Посмотрим на соотношение классов. Как и ожидалось, токсичные комментарии составляют меньшинство (около 10%):

In [4]:
print('Соотношение классов в процентах:')
df['toxic'].value_counts(normalize=True).round(3)*100

Соотношение классов в процентах:


toxic
0    89.8
1    10.2
Name: proportion, dtype: float64

Теперь применим функции лемматизации и очистки текста, а затем создадим TfIdf вектора (с удалением стопслов) и 2-граммы.

In [5]:
tag_dict = {"J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV}
wnl = WordNetLemmatizer()


def get_wordnet_pos(word):
    return tag_dict.get(nltk.pos_tag([word])[0][1][0].upper(), wordnet.NOUN)


def lemmatize(word_list):
    return [wnl.lemmatize(w, get_wordnet_pos(w)) for w in word_list]


def clear_text(text):
    return ' '.join(lemmatize(re.sub(r'[^A-Za-z\' ]', ' ', text).split()))




In [6]:
[x**2 for x in[]]

[]

In [7]:
processed = True
filename = 'cleared_text.csv'
if not processed:
    df['text'] = df['text'].progress_apply(lambda x: clear_text(x))
    df.to_csv(filename)
else:
    df = check_and_read(filename, 0).fillna('')

Посмотрим на полученный результат:

In [8]:
df.head()

,text,toxic
0,Explanation Why the edits make under my userna...,0
1,D'aww He match this background colour I'm seem...,0
2,Hey man I'm really not try to edit war It's ju...,0
3,More I can't make any real suggestion on impro...,0
4,You sir be my hero Any chance you remember wha...,0


## Обучение

Выделим тренировочный и тестовый датасеты:

In [9]:
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['toxic'], random_state=RANDOM_STATE)

Препроцессинг в виде масштабирования признаков обычно не проводится для TfIdf векторов. Применим логистическую регрессию, как наименее затратную по ресурсам, изменяя параметр C и балансируя веса классов.

In [10]:
stop_words = stopwords.words('english')


pipe_final = Pipeline([
    ('preprocessor', TfidfVectorizer(stop_words = stop_words, ngram_range=(1, 2))),
    ('models', 'passthrough')
])

param_grid = [
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)],
        'models__C': [10, 20, 30],
        'models__class_weight': ['balanced', None],
    },
]

In [11]:
use_prev = False
filename = 'final_model_text.sav'
if use_prev:
    hyper_search = pickle.load(open(filename, 'rb'))
else:
    hyper_search = GridSearchCV(
        pipe_final,
        param_grid,
        cv=5,
        scoring='f1',
        verbose=3,
        n_jobs=-1,
        return_train_score=True,
    )
    hyper_search.fit(X_train, y_train)
    pickle.dump(hyper_search, open(filename, 'wb'))

print('Лучшая модель и её параметры:\n\n', hyper_search.best_estimator_)
print(hyper_search.best_params_)
print(f'Метрика F1 лучшей модели на тренировочной выборке: {hyper_search.best_score_:.3f}')

y_pred = hyper_search.predict(X_test)
print('Сравнение первых 10 предсказаний и истинных значений:')
print(y_pred[:10])
print(y_test[:10].to_numpy())
print(f'Метрика F1 на тестовой выборке: {f1_score(y_test, y_pred):.3f}')
print(f'Метрика precision на тестовой выборке: {precision_score(y_test, y_pred):.3f}')
print(f'Метрика recall на тестовой выборке: {recall_score(y_test, y_pred):.3f}')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшая модель и её параметры:

 Pipeline(steps=[('preprocessor',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('models',
                 LogisticRegression(C=30, class_weight='balanced',
                                    max_iter=1000, random_state=42))])
{'models': LogisticRegression(max_i

Полученный результат: F1 = 0,781 (precision = 0,754; recall = 0,809). Выявленные оптимальные гиперпараметры: C = 30, с балансированием весов классов. Результат удовлетворительный.

## Выводы

В ходе работы были загружены исходные данные, созданы вектора TfIdf, построена модель логистической регрессии. Лемматизация датасета заняла порядка 20 минут.  
Полученный на итоговой модели результат: F1 = 0,782 (precision = 0,786; recall = 0,778). Это превышает требуемый порог 0,75 для F1 меры и соответствует поставленной задаче. Выявленные оптимальные гиперпараметры модели: логистическая регрессия, C = 20, с балансированием весов классов.